In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.metrics import  roc_auc_score

import pickle 
import logging


# from config.config import SQLQuery
# querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
from preprocess import Convert,MissingValues,Outlier,FeatureSelection
from feature_transformation import Scaler,Transform,Selection
from model_building import split_test_train, feature_encoding, classification_models
from model_evaluations import model_metrics, feature_importance, probability_bins, cross_validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor

# object initiation 
tf = Transform()
sel = Selection()
ft = FeatureSelection()
cv = Convert()
mv = MissingValues()
ot = Outlier()

# set seed
seed = 1

In [3]:
corr_arr = 0.4 # person correlation coefficient # change it to 0.5
vif_arr = 5 # vif coefficient
features_arr = 10 # total number of features to be selected from backward feature selection
iv_upper_limit = 0.5 # upper threshold of iv # change it to 0.6
iv_lower_limit = 0.02 # lower threshold of iv

In [4]:
df_raw = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/final_dataset_2.pkl')
df_raw.shape

(646, 255)

In [5]:
df_raw['txn_flag'].value_counts()

txn_flag
1.0    637
0.0      9
Name: count, dtype: int64

In [6]:
df_raw = df_raw[df_raw['txn_flag']==1]
df_raw.shape

(637, 255)

In [7]:
df_raw = df_raw.drop(['business_id', 'lending_business_id','decision_date','drawn_flag', 'everDPD_15', 'fico_score','txn_flag'], axis=1)
df_raw.shape

(637, 248)

In [24]:
df_raw = df_raw.fillna(0)

In [ ]:
res = pd.DataFrame(columns = ['ind','mean_train_auc', 'mean_test_auc', 'train_ro', 'test_ro'])

In [275]:
ind = 0

feat_list = ['credit_size_3M',
 'pos_size_1M',
 'ratio_amt_credits_less_100_credits_grt_500_3M',
 'median_credit_amt_1M',
 'ratio_pos_size_1M_6M',
 'median_pos_amt_3M',
 'count_credits_grt_500_3M',
 'loans_size_6M',
 'ach_count_6M']

In [276]:
## Model Evaluation - KS & ROC AUC

def ks(target=None, prob=None):
    data = pd.DataFrame()
    data['y'] = target
    data['y'] = data['y'].astype(float)
    data['p'] = prob
    data['y0'] = 1- data['y']
    data['bucket'] = pd.qcut(data['p'], 5, duplicates = 'drop')
    grouped = data.groupby('bucket', as_index=False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()['p']
    kstable['max_prob'] = grouped.max()['p']
    kstable['events'] = grouped.sum()['y']
    kstable['nonevents'] = grouped.sum()['y0']
    kstable = kstable.sort_values(by='min_prob', ascending=False).reset_index(drop=True)
    kstable['event_rate'] = (kstable.events / data['y'].sum()).apply('{0:.2%}'.format)
    kstable['nonevent_rate'] = (kstable['nonevents'] /  data['y0'].sum()).apply('{0:2%}'.format)
    kstable['cum_eventrate'] = (kstable.events / data['y'].sum()).cumsum()
    kstable['cum_noneventrate'] = (kstable.nonevents / data['y0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate'] - kstable['cum_noneventrate'], 3) * 100
    kstable['bad_rate'] = (kstable['events'] / (kstable['events'] + kstable['nonevents'])) * 100
    
    # formatting
    kstable['cum_eventrate'] = kstable['cum_eventrate'].apply('{0:.2%}'.format)
    kstable['cum_noneventrate'] = kstable['cum_noneventrate'].apply('{0:.2%}'.format)
    # kstable.index = range(1,len(data['bucket'].unique())+1)
    # kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    # Display KS
    # print("KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return kstable

In [277]:
def is_column_descending(df, column_name):
    is_descending = df[column_name].is_monotonic_decreasing
    return int(is_descending)

In [278]:
## optimal binning woe

import optbinning as optb
from optbinning import Scorecard, BinningProcess, OptimalBinning
from optbinning.binning.binning_statistics import BinningTable

res_df = pd.DataFrame(columns = ['seed', 'train_auc', 'test_auc', 'train_ro', 'test_ro'])
for seed in range(50):
    # train test split
    x_train, y_train, x_test, y_test = split_test_train(df_raw, target_column='target', test_size=0.3, random_state=seed)
    
    # copy to df
    df = x_train.copy(deep=True)
    df['target'] = df_raw['target']

    df_temp = df.copy()
    df_temp['target'] = df['target']
    df_temp = df_temp.fillna(0)

    transformed_vars = df.copy()

    # 1
    # transform
    col         = 'stddev_credit_amt_3M'
    conditions  = [ transformed_vars[col] <= 0, 
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 578), 
                (transformed_vars[col] >= 578 ) & (transformed_vars[col] < 1274), 
                (transformed_vars[col] >= 1274) & (transformed_vars[col] < 7646),
                transformed_vars[col] >= 7646 ]

    choices     = [0.090405,-0.551449,-0.013445, 0.40452,1.951157]
    
    transformed_vars["stddev_credit_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 2
    # transform
    col         = 'shops_amt_3M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 221.5), 
                (transformed_vars[col] >= 221.5 ) & (transformed_vars[col] < 2588), 
                transformed_vars[col] >= 2588 ]

    choices     = [0.226839,-0.738544,-0.408586,0.38456]
    
    transformed_vars["shops_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 3
    # transform
    col         = 'pos_size_1M'
    conditions  = [ transformed_vars[col] < 235.11, 
                (transformed_vars[col] >= 235.11 ) & (transformed_vars[col] < 979.44), 
                transformed_vars[col] >= 979.44 ]

    choices     = [-0.152312,0.159398,1.699843]
    
    transformed_vars["pos_size_1M"] = np.select(conditions, choices, default=np.nan)

    # 4
    # transform
    col         = 'loans_amt_6M'
    conditions  = [ transformed_vars[col] <= 0, 
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 21503),
                transformed_vars[col] >= 21503 ]

    choices     = [0.08126,-0.518001,0.647751]
    
    transformed_vars["loans_amt_6M"] = np.select(conditions, choices, default=np.nan)

    # 5
    # transform
    col         = 'ratio_amt_credits_less_100_credits_grt_500_3M'
    conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

    choices     = [-0.112536,0.045514]
    
    transformed_vars["ratio_amt_credits_less_100_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

    # 6
    # transform
    ol         = 'median_credit_amt_1M'
    conditions  = [ transformed_vars[col] < 102,
                (transformed_vars[col] >= 102 ) & (transformed_vars[col] < 475.2),
                transformed_vars[col] >= 475.2 ]

    choices     = [-0.444738,-0.119068,0.550264]
    
    transformed_vars["median_credit_amt_1M"] = np.select(conditions, choices, default=np.nan)

    # 7
    # transform
    col         = 'ratio_pos_size_1M_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.21),
                transformed_vars[col] >= 1.21 ]

    choices     = [-0.164722,0.141314,1.140227]
    
    transformed_vars["ratio_pos_size_1M_6M"] = np.select(conditions, choices, default=np.nan)

    # 8
    # transform
    col         = 'median_ecom_amt_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 49.74),
                transformed_vars[col] >= 49.74 ]

    choices     = [0.30558,-0.576309,-0.303226]
    
    transformed_vars["median_ecom_amt_6M"] = np.select(conditions, choices, default=np.nan)

    # 9
    # transform
    col         = 'pos_count_3M'
    conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

    choices     = [-0.25334,0.201958]
    
    transformed_vars["pos_count_3M"] = np.select(conditions, choices, default=np.nan)

    # 10
    # transform
    col         = 'median_pos_amt_3M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 486.25),
                transformed_vars[col] >= 486.25 ]

    choices     = [-0.25334,-0.127284,0.968955]
    
    transformed_vars["median_pos_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 11
    # transform
    col         = 'count_credits_grt_500_3M'
    conditions  = [ transformed_vars[col] < 1.5,
                (transformed_vars[col] >= 1.5 ) & (transformed_vars[col] < 15.5),
                transformed_vars[col] >= 15.5 ]

    choices     = [-0.450862,-0.000945,0.284561]
    
    transformed_vars["count_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

    # 12
    # transform
    col         = 'loans_size_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 3317),
                transformed_vars[col] >= 3317 ]

    choices     = [0.08126,-0.605012,0.814805]
    
    transformed_vars["loans_size_6M"] = np.select(conditions, choices, default=np.nan)

    # 13
    # transform
    col         = 'ratio_loans_size_credit_size_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.11),
                transformed_vars[col] >= 1.11 ]

    choices     = [0.08126,-0.389168,-0.107917]
    
    transformed_vars["ratio_loans_size_credit_size_6M"] = np.select(conditions, choices, default=np.nan)

    # 14
    # transform
    col         = 'sum_credits_grt_500_1M'
    conditions  = [ transformed_vars[col] < 6587,
                transformed_vars[col] >= 6587 ]

    choices     = [-0.246067,0.422079]
    
    transformed_vars["sum_credits_grt_500_1M"] = np.select(conditions, choices, default=np.nan)

    # 15
    # transform
    col         = 'ach_count_6M'
    conditions  = [ transformed_vars[col] < 6.5,
                transformed_vars[col] >= 6.5 ]

    choices     = [-0.022101,0.149246]
    
    transformed_vars["ach_count_6M"] = np.select(conditions, choices, default=np.nan)

    # 16
    # transform
    col         = 'credit_size_3M'
    conditions  = [ transformed_vars[col] < 178.8,
                (transformed_vars[col] >= 178.9 ) & (transformed_vars[col] < 392.56),
                (transformed_vars[col] >= 392.56 ) & (transformed_vars[col] < 2274.8),
                transformed_vars[col] >= 2274.8 ]

    choices     = [-1.056997,-0.484478,0.184337,1.294378]
    
    transformed_vars["credit_size_3M"] = np.select(conditions, choices, default=np.nan)



    # copy vars
    X_train = transformed_vars.copy()

    # hyperparameters
    params_log_reg = {'penalty': 'l2',
                  'random_state': seed,
                  'solver': 'liblinear',
                  'class_weight': 'balanced'}

    # model fit
    logreg_model = classification_models(X_train[feat_list], y_train, params_log_reg, models=['log_reg'])

    # binning variable transform
    x_test.fillna(0, inplace=True)

    transformed_vars = x_test

    # 1
    # transform
    col         = 'stddev_credit_amt_3M'
    conditions  = [ transformed_vars[col] <= 0, 
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 578), 
                (transformed_vars[col] >= 578 ) & (transformed_vars[col] < 1274), 
                (transformed_vars[col] >= 1274) & (transformed_vars[col] < 7646),
                transformed_vars[col] >= 7646 ]

    choices     = [0.090405,-0.551449,-0.013445, 0.40452,1.951157]
    
    transformed_vars["stddev_credit_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 2
    # transform
    col         = 'shops_amt_3M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 221.5), 
                (transformed_vars[col] >= 221.5 ) & (transformed_vars[col] < 2588), 
                transformed_vars[col] >= 2588 ]

    choices     = [0.226839,-0.738544,-0.408586,0.38456]
    
    transformed_vars["shops_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 3
    # transform
    col         = 'pos_size_1M'
    conditions  = [ transformed_vars[col] < 235.11, 
                (transformed_vars[col] >= 235.11 ) & (transformed_vars[col] < 979.44), 
                transformed_vars[col] >= 979.44 ]

    choices     = [-0.152312,0.159398,1.699843]
    
    transformed_vars["pos_size_1M"] = np.select(conditions, choices, default=np.nan)

    # 4
    # transform
    col         = 'loans_amt_6M'
    conditions  = [ transformed_vars[col] <= 0, 
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 21503),
                transformed_vars[col] >= 21503 ]

    choices     = [0.08126,-0.518001,0.647751]
    
    transformed_vars["loans_amt_6M"] = np.select(conditions, choices, default=np.nan)

    # 5
    # transform
    col         = 'ratio_amt_credits_less_100_credits_grt_500_3M'
    conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

    choices     = [-0.112536,0.045514]
    
    transformed_vars["ratio_amt_credits_less_100_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

    # 6
    # transform
    ol         = 'median_credit_amt_1M'
    conditions  = [ transformed_vars[col] < 102,
                (transformed_vars[col] >= 102 ) & (transformed_vars[col] < 475.2),
                transformed_vars[col] >= 475.2 ]

    choices     = [-0.444738,-0.119068,0.550264]
    
    transformed_vars["median_credit_amt_1M"] = np.select(conditions, choices, default=np.nan)

    # 7
    # transform
    col         = 'ratio_pos_size_1M_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.21),
                transformed_vars[col] >= 1.21 ]

    choices     = [-0.164722,0.141314,1.140227]
    
    transformed_vars["ratio_pos_size_1M_6M"] = np.select(conditions, choices, default=np.nan)

    # 8
    # transform
    col         = 'median_ecom_amt_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 49.74),
                transformed_vars[col] >= 49.74 ]

    choices     = [0.30558,-0.576309,-0.303226]
    
    transformed_vars["median_ecom_amt_6M"] = np.select(conditions, choices, default=np.nan)

    # 9
    # transform
    col         = 'pos_count_3M'
    conditions  = [ transformed_vars[col] <= 0,
                transformed_vars[col] > 0 ]

    choices     = [-0.25334,0.201958]
    
    transformed_vars["pos_count_3M"] = np.select(conditions, choices, default=np.nan)

    # 10
    # transform
    col         = 'median_pos_amt_3M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 486.25),
                transformed_vars[col] >= 486.25 ]

    choices     = [-0.25334,-0.127284,0.968955]
    
    transformed_vars["median_pos_amt_3M"] = np.select(conditions, choices, default=np.nan)

    # 11
    # transform
    col         = 'count_credits_grt_500_3M'
    conditions  = [ transformed_vars[col] < 1.5,
                (transformed_vars[col] >= 1.5 ) & (transformed_vars[col] < 15.5),
                transformed_vars[col] >= 15.5 ]

    choices     = [-0.450862,-0.000945,0.284561]
    
    transformed_vars["count_credits_grt_500_3M"] = np.select(conditions, choices, default=np.nan)

    # 12
    # transform
    col         = 'loans_size_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 3317),
                transformed_vars[col] >= 3317 ]

    choices     = [0.08126,-0.605012,0.814805]
    
    transformed_vars["loans_size_6M"] = np.select(conditions, choices, default=np.nan)

    # 13
    # transform
    col         = 'ratio_loans_size_credit_size_6M'
    conditions  = [ transformed_vars[col] <= 0,
                (transformed_vars[col] > 0 ) & (transformed_vars[col] < 1.11),
                transformed_vars[col] >= 1.11 ]

    choices     = [0.08126,-0.389168,-0.107917]
    
    transformed_vars["ratio_loans_size_credit_size_6M"] = np.select(conditions, choices, default=np.nan)

    # 14
    # transform
    col         = 'sum_credits_grt_500_1M'
    conditions  = [ transformed_vars[col] < 6587,
                transformed_vars[col] >= 6587 ]

    choices     = [-0.246067,0.422079]
    
    transformed_vars["sum_credits_grt_500_1M"] = np.select(conditions, choices, default=np.nan)

    # 15
    # transform
    col         = 'ach_count_6M'
    conditions  = [ transformed_vars[col] < 6.5,
                transformed_vars[col] >= 6.5 ]

    choices     = [-0.022101,0.149246]
    
    transformed_vars["ach_count_6M"] = np.select(conditions, choices, default=np.nan)

    # 16
    # transform
    col         = 'credit_size_3M'
    conditions  = [ transformed_vars[col] < 178.8,
                (transformed_vars[col] >= 178.9 ) & (transformed_vars[col] < 392.56),
                (transformed_vars[col] >= 392.56 ) & (transformed_vars[col] < 2274.8),
                transformed_vars[col] >= 2274.8 ]

    choices     = [-1.056997,-0.484478,0.184337,1.294378]
    
    transformed_vars["credit_size_3M"] = np.select(conditions, choices, default=np.nan)

    # copy
    X_test = transformed_vars.copy()

    # predicted proability
    train_pred = logreg_model.predict_proba(X_train[feat_list])[:,1]
                                                                     
    test_pred = logreg_model.predict_proba(X_test[feat_list])[:,1]


    train_ks = ks(y_train, train_pred)
    test_ks = ks(y_test, test_pred)


    train_auc = roc_auc_score(y_train, train_pred)
    test_auc = roc_auc_score(y_test, test_pred)

    train_ro = is_column_descending(train_ks, 'bad_rate')
    test_ro = is_column_descending(test_ks, 'bad_rate')
    

    temp = {'seed' : [seed],
            'train_auc' : [train_auc],
            'test_auc' : [test_auc],
            'train_ro' : [train_ro],
            'test_ro' : [test_ro]}

    res_df = pd.concat([res_df, pd.DataFrame(temp)], ignore_index = True)
    

In [284]:
res_df[(res_df['train_ro']==1) & (res_df['test_ro']==1)]

,seed,train_auc,test_auc,train_ro,test_ro
14,14,0.692065,0.741694,1,1
16,16,0.698855,0.727467,1,1
24,24,0.691603,0.729112,1,1
33,33,0.688308,0.729934,1,1
34,34,0.708338,0.675658,1,1
37,37,0.710186,0.710938,1,1
42,42,0.694528,0.728043,1,1


In [279]:
mean_train_auc = [res_df['train_auc'].mean()]
mean_test_auc = [res_df['test_auc'].mean()]
train_ro = [res_df['train_ro'].sum()/50]
test_ro = [res_df['test_ro'].sum()/50]

In [280]:
t = {'ind' : ind,
    'mean_train_auc' : mean_train_auc,
    'mean_test_auc' : mean_test_auc,
    'train_ro' : train_ro,
    'test_ro' : test_ro}

In [281]:
res = pd.concat([res, pd.DataFrame(t)], ignore_index = True)

In [282]:
res.sort_values(by='test_ro', ascending=False)

,ind,mean_train_auc,mean_test_auc,train_ro,test_ro
20,7468,0.697192,0.653046,0.78,0.42
15,4633,0.696853,0.653697,0.72,0.36
9,4624,0.697357,0.654113,0.84,0.34
11,4367,0.689421,0.646891,0.76,0.32
8,1014,0.696289,0.658097,0.76,0.30
17,9070,0.698035,0.649801,0.78,0.28
4,7460,0.698113,0.649834,0.84,0.28
0,925,0.684118,0.641752,0.66,0.26
29,8955,0.697032,0.643169,0.80,0.26
3,4636,0.696693,0.653462,0.68,0.26
